In [ ]:
!pip install python-dotenv

In [1]:
!pip install xmltodict

In [22]:
import os
import requests
import xmltodict
import pandas as pd
from dotenv import load_dotenv

# ===============================
# 1. 환경변수 로드
# ===============================
load_dotenv()
API_KEY = os.getenv("KIPRIS_API_KEY")

if not API_KEY:
    raise RuntimeError("KIPRIS_API_KEY가 설정되어 있지 않습니다.")

# ===============================
# 2. Endpoint
# ===============================
BASE_URL = "http://plus.kipris.or.kr/kipo-api/kipi/patUtiModInfoSearchSevice/getAdvancedSearch"

# ===============================
# 3. 검색 조건
# ===============================
COMPANY_NAME = "터보파워텍"

params = {
    "ServiceKey": API_KEY,
    "applicant": COMPANY_NAME,
    "patent": "true",
    "utility": "true",
    "numOfRows": 500,
    "pageNo": 1
}

# ===============================
# 4. API 요청
# ===============================
response = requests.get(BASE_URL, params=params, timeout=10)

# HTTP 자체 실패
if response.status_code != 200:
    raise RuntimeError(
        f"HTTP 오류 발생: {response.status_code}\n{response.text}"
    )

# ===============================
# 5. XML 파싱
# ===============================
try:
    data = xmltodict.parse(response.text)
except Exception:
    raise RuntimeError(
        "XML 파싱 실패. 응답이 XML이 아닙니다.\n"
        f"응답 원문:\n{response.text}"
    )

# ===============================
# 6. API 헤더 에러 검사 (🔥 핵심)
# ===============================
header = data.get("response", {}).get("header", {})

result_code = header.get("resultCode")
result_msg = header.get("resultMsg")

if result_code and result_code != "00":
    raise RuntimeError(
        f"KIPRIS API 오류\n"
        f"resultCode: {result_code}\n"
        f"resultMsg : {result_msg}"
    )

# ===============================
# 7. body / items 검사
# ===============================
body = data.get("response", {}).get("body")

if not body:
    raise RuntimeError(
        "API body가 없습니다. 검색 조건이 인식되지 않았습니다."
    )

items = body.get("items", {}).get("item")

if not items:
    raise RuntimeError(
        f"검색 결과 없음.\n"
        f"출원인명 '{COMPANY_NAME}'이(가) "
        f"현재 파라미터로는 인식되지 않았습니다.\n\n"
        f"⚠️ applicant 파라미터가 무시되었을 가능성이 큽니다."
    )

# ===============================
# 8. DataFrame 변환
# ===============================
if isinstance(items, dict):
    items = [items]

df = pd.DataFrame(items)

# print("컬럼 목록:", df.columns.tolist())
# print(df.head())
# print(f"\n총 {len(df)}건")
df.head(5)

,applicantName,applicationDate,applicationNumber,astrtCont,bigDrawing,drawing,indexNo,inventionTitle,ipcNumber,openDate,openNumber,publicationDate,publicationNumber,registerDate,registerNumber,registerStatus
0,터보파워텍(주),20071127,1020070121426,발전 터빈기기에 장착되는 다이아프램 제작방법 및 그 방법에 의해 제작된 다이아프램을...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,1,터빈용 다이아프램 제작방법 및 그 방법에 의해 제작된터빈용 다이아프램,F01D 25/24|F01D 9/02|F01D 5/28,None,None,20081231,None,20081223,1008766030000,등록
1,터보파워텍(주)|한국서부발전 주식회사,20210825,1020210112331,"검사정밀성이 개선되도록, 본 발명은 중앙부에 상부 및 전방이 개구되는 장착홈이 형성...",http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,2,베인 검사장치,G01M 7/08|G01M 7/02|G01N 29/04|B25B 11/00,None,None,20211228,None,20211222,1023438080000,등록
2,터보파워텍(주),20220902,1020220111724,"본 발명은 터빈로터 레이저 클래딩 수리장치에 관한 것으로, 더욱 상세하게는 터빈로터...",http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,3,터빈로터 수리용 레이저 클래딩 자동화장치,B23P 6/00|B23K 37/04|B23K 37/02,None,None,20221223,None,20221219,1024803170000,등록
3,터보파워텍(주),20080401,1020080030414,본 발명의 일 실시예에 따른 터빈용 노멀 파티션의 제조 방법은 링 형상의 합금 모재...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,4,터빈용 노멀 파티션의 제조 방법,F01D 25/24|B22D 13/04|F01D 9/04,None,None,20081222,None,20081216,1008755150000,등록
4,터보파워텍(주),20250314,1020250033081,"정밀한 가압주조를 통해 조직의 치밀도를 균일하게 향상시키도록, 본 발명은 하단부에 ...",http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,5,가압주조를 이용한 터빈용 씰링부재의 제조장치,B22D 18/02|B22D 27/11|F01D 11/08,None,None,20251204,None,20251201,1028958930000,등록
